# Getting Started -- Lean

* make sure jupyter server proxy is installed
* copy httpserver.py and LeanGettingStarted.ipynb in the same directory of a Jupyter Lab server
* create a widgets subdirectory there
* Run the notbook (Kernel -> Restart Kernel And Run All Cells...)

you should get the url of this notebook displayed in the last cell

## Libraries

In [1]:
from httpserver import start, stop
from http.server import SimpleHTTPRequestHandler, HTTPServer
import os
from urllib.parse import urlparse, parse_qs
from IPython.display import clear_output, HTML
import json
import ipywidgets as widgets
import time

## Backend: the HTTP server

### Widgets python implementation

In [2]:
logsw=widgets.Textarea(layout=widgets.Layout(width='50%'))
logsw.value = ""
bag ={}
class myHTTPRequestHandler(SimpleHTTPRequestHandler):
    callback={}
    def __init__(self, *args, directory=None,bag=bag, **kwargs):
        self.bag = bag
        self.directory = os.path.join(os.getcwd(),"widgets")
        super().__init__(*args, directory=self.directory, **kwargs)
        # print(self.directory)
    def end_headers(self):
        super().end_headers()
    def do_GET(self):
        self.parsed_path = urlparse(self.path)
        self.queryparams = parse_qs(self.parsed_path.query)
        if self.path.endswith('/version'):
            self.version()
        elif self.parsed_path.path.endswith('/setvalue'):
            self.setvalue()
        else:
            super().do_GET()
    def version(self):
        ans = '{"version": "0.0"}'
        eans = ans.encode()        
        self.send_response(200)
        self.send_header("Content-type", "application/json")
        self.send_header("Content-Length",len(eans))
        self.end_headers()
        self.wfile.write(eans)
    def setvalue(self):
        self.bag.update({self.queryparams['variable'][0]:self.queryparams['value'][0]})
        if self.queryparams['variable'][0] in myHTTPRequestHandler.callback:
            self.callback[self.queryparams['variable'][0]](self.queryparams['value'][0])
        self.version() 
    def log_message(self, format, *args):
        global logsw
        v = format % args
        t = time.localtime()
        current_time = time.strftime("%H:%M:%S", t)
        logsw.value += current_time + " " + v +"\n"
logsw

Output()

In [3]:
start(handler_class=myHTTPRequestHandler, port=8085)

Starting httpd...



## Frontend

### Widget javascript implementation

In [4]:
HTML("<script>fetch( window.location.origin+'/proxy/8085/setvalue?value='+encodeURIComponent(window.location)+'&variable=url')</script>")

In [5]:
while 'url' not in bag:
    time.sleep(0.1)
bag

{'url': 'http://raspberry0:8888/lab/tree/Labwidgets/LeanGettingStarted.ipynb'}

In [6]:
stop()